todo: 
    
* clear data after every read, and update-the plot-data-source accordingly
* rewrite markdown
* checkin in new repo
* refactor count_red_pixels and set_rates_from_ratios to one single helperfunction in the brain

(ignore this cell)

todo...


In [ ]:
import time

import numpy as np
import pybullet as p
import pybullet_data

import pyNN.nest as sim
from pyNN.utility import get_simulator, normalized_filename

..setup up brain sim with pynn

todo: link to hbrp

In [ ]:
# setup neurosim with pynn

def get_pop():

    SENSORPARAMS = {'v_rest': -60.5,
                    'cm': 0.025,
                    'tau_m': 10.,
                    'tau_refrac': 10.0,
                    'tau_syn_E': 2.5,
                    'tau_syn_I': 2.5,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -60.0,
                    'v_reset': -60.5}

    GO_ON_PARAMS = {'v_rest': -60.5,
                    'cm': 0.025,
                    'tau_m': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_reset': -61.6,
                    'v_thresh': -60.51,
                    'tau_refrac': 10.0,
                    'tau_syn_E': 2.5,
                    'tau_syn_I': 2.5}

    population = sim.Population(8, sim.IF_cond_alpha())
    population[0:5].set(**SENSORPARAMS)
    population[5:6].set(**GO_ON_PARAMS)
    population[6:8].set(**SENSORPARAMS)

    syn_params = {'U': 1.0, 'tau_rec': 1.0, 'tau_facil': 1.0}

    # Synaptic weights
    WEIGHT_RED_TO_ACTOR = 1.5e-4
    WEIGHT_RED_TO_GO_ON = 1.2e-3  # or -1.2e-3?
    WEIGHT_GREEN_BLUE_TO_ACTOR = 1.05e-4
    WEIGHT_GO_ON_TO_RIGHT_ACTOR = 1.4e-4
    DELAY = 1

    # Connect neurons
    CIRCUIT = population

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_RED_TO_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[2:3],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')
    sim.Projection(presynaptic_population=CIRCUIT[3:4],
                   postsynaptic_population=CIRCUIT[6:7],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')


    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_RED_TO_GO_ON),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[0:2],
                   postsynaptic_population=CIRCUIT[4:5],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='inhibitory')
    sim.Projection(presynaptic_population=CIRCUIT[0:2],
                   postsynaptic_population=CIRCUIT[5:6],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='inhibitory')

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_GREEN_BLUE_TO_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[4:5],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_GO_ON_TO_RIGHT_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[5:6],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    return population    




spike_generator...

link to hbpr

In [ ]:


class spike_generator(object):
    """the class handles the input for the brain"""
    min_rate = 0.0005
    def __init__(self, population, interval=20.0, count_red_left=1/3, count_red_right=1/3, count_non_red=1/3):
        params_source=  {'start': 0.0, 'duration': float("inf"), 'rate': 0.0}
        params_connec= {'weight':0.00015, 'receptor_type':'excitatory','delay':0.1}
        self.ssp_red_left_eye = sim.create(sim.SpikeSourcePoisson,  params_source)
        self.ssp_red_right_eye = sim.create(sim.SpikeSourcePoisson,  params_source)
        self.ssp_green_blue_eye = sim.create(sim.SpikeSourcePoisson,  params_source)
        sim.connect(self.ssp_red_left_eye, population[slice(0, 3, 2)], **params_connec)
        sim.connect(self.ssp_red_right_eye, population[slice(1, 4, 2)], **params_connec)
        sim.connect(self.ssp_green_blue_eye, population[4], **params_connec)
    
    def set_rates_from_ratios(self, count_red_left, count_red_right, count_non_red):
        if count_red_left <= self.min_rate:
            count_red_left = self.min_rate
        if count_red_right <= self.min_rate:
            count_red_right = self.min_rate
        if count_non_red <= self.min_rate:
            count_non_red = self.min_rate
        try:
            self.ssp_red_left_eye.set(rate=2*2000.0 * count_red_left)
            self.ssp_red_right_eye.set(rate=2*2000.0 * count_red_right)
            self.ssp_green_blue_eye.set(rate=75.0 * count_non_red)
        except StopIteration:
            pass

output_generator...

In [ ]:

class output_generator(object):
    def __init__(self, population):
        """this class handles the output of the brain
        it connects two leaky-integrator neurons on the output-neurons to convert
        their spikes into voltages, which we will measure. 
        
        parameters taken from:
https://developer.humanbrainproject.eu/docs/projects/hbp-nrp-cle/1.3.8/
modules/hbp_nrp_cle/brainsim/pynn/devices/__PyNNLeakyIntegratorTypes.html
"""
        LI_create = {'v_thresh': float('inf'),'cm': 1.0,'tau_m': 10.0,'tau_syn_E': 2.,
            'tau_syn_I': 2.,'v_rest': 0.0,'v_reset': 0.0,'tau_refrac': 0.1,}
        LI_connect = {'weight':0.00015, 'receptor_type':'excitatory','delay':0.1}
        self.left = sim.create(sim.IF_curr_alpha, LI_create)
        self.right = sim.create(sim.IF_curr_alpha, LI_create)
        sim.connect(population[6], self.left, **LI_connect)
        sim.connect(population[7],self.right,  **LI_connect)
        sim.initialize(self.left, v=self.left.get('v_rest'))
        sim.initialize(self.right, v=self.right.get('v_rest'))
        self.left.record('v')
        self.right.record('v')
    
    def get_current_voltage(self):
        return self._get_current_voltage(self.left), self._get_current_voltage(self.right)
    def _get_current_voltage(self, cell):
        return cell.get_data('v', clear=False).segments[0].filter(name="v")[0][-1,0]



World...

(explanation about how the world looks and what it does)

In [ ]:
#setup physics sim with pybullet

class World(object):
    """this class handles the world sans robot
    the world is just a plane, with two rectangles, which switch colors every 5000 steps"""
    green = [0, 1, 0, 1]
    blue = [0, 0, 1, 1]
    red = [1, 0, 0, 1]
    count_steps = 0
    count_switch = 0

    def __init__(self):
        p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
        p.setGravity(0, 0, -10)
        p.loadURDF("plane.urdf")
        plane_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[1, 0.25, 1])
        plane_vis_id = p.createVisualShape(p.GEOM_BOX, halfExtents=[1, 0.25, 1], rgbaColor=self.blue)
        self.planeA = p.createMultiBody(baseCollisionShapeIndex=plane_id, basePosition=[0, 5, 2],
                                        baseVisualShapeIndex=plane_vis_id)
        self.planeB = p.createMultiBody(baseCollisionShapeIndex=plane_id, basePosition=[0, -5, 2],
                                        baseVisualShapeIndex=plane_vis_id)
        p.setRealTimeSimulation(0)

    def do_step(self):
        self.count_steps = self.count_steps + 1
        if self.count_steps % 5000 == 1:
            self.count_switch = self.count_switch + 1
            if self.count_switch % 2 == 0:
                p.changeVisualShape(self.planeA, -1, rgbaColor=self.green)
                p.changeVisualShape(self.planeB, -1, rgbaColor=self.red)
            else:
                p.changeVisualShape(self.planeA, -1, rgbaColor=self.red)
                p.changeVisualShape(self.planeB, -1, rgbaColor=self.green)
        p.stepSimulation()



body for robot

(explanation about why husky, and how it was modified (camera, motorcontrol))

In [ ]:

class Husky(object):
    maxForce = 0
    targetVelocity = 0

    class Camera(object):
        init_camera_vector = (1, 0, 0)
        init_up_vector = (0, 0, 1)

        def __init__(self, pixel_width=80, pixel_height=80, near_plane=0.1, far_plane=100, fov=100):
            self.pixel_width = pixel_width
            self.pixel_height = pixel_height
            self.projection_mat = p.computeProjectionMatrixFOV(fov, pixel_width / pixel_height, near_plane, far_plane)

    def __init__(self):
        self.camera = Husky.Camera()
        self.husky_model = p.loadURDF("husky/husky.urdf",
                                      basePosition=[0, 0, 1],
                                      baseOrientation=p.getQuaternionFromEuler([0, 0, 0]))

        for joint in range(p.getNumJoints(self.husky_model)):
            # print("joint[", joint, "]=", p.getJointInfo(self.husky_model, joint))
            if p.getJointInfo(self.husky_model, joint)[1] == b'user_rail':
                self.zed_camera_joint = joint
            if p.getJointInfo(self.husky_model, joint)[1] == b'front_left_wheel':
                self.front_left_wheel = joint
            if p.getJointInfo(self.husky_model, joint)[1] == b'front_right_wheel':
                self.front_right_wheel = joint
            if p.getJointInfo(self.husky_model, joint)[1] == b'rear_left_wheel':
                self.rear_left_wheel = joint
            if p.getJointInfo(self.husky_model, joint)[1] == b'rear_right_wheel':
                self.rear_right_wheel = joint

    def update_control(self, command_left, command_right):
        p.setJointMotorControl2(self.husky_model, self.front_left_wheel, p.VELOCITY_CONTROL,
                                targetVelocity=command_left * self.targetVelocity,
                                force=self.maxForce)
        p.setJointMotorControl2(self.husky_model, self.rear_left_wheel, p.VELOCITY_CONTROL,
                                targetVelocity=command_left * self.targetVelocity,
                                force=self.maxForce)
        p.setJointMotorControl2(self.husky_model, self.front_right_wheel, p.VELOCITY_CONTROL,
                                targetVelocity=command_right * self.targetVelocity,
                                force=self.maxForce)
        p.setJointMotorControl2(self.husky_model, self.rear_right_wheel, p.VELOCITY_CONTROL,
                                targetVelocity=command_right * self.targetVelocity,
                                force=self.maxForce)

    def update_cam(self):
        joint_pos, joint_orn, _, _, _, _ = p.getLinkState(self.husky_model, self.zed_camera_joint,
                                                          computeForwardKinematics=True)
        cam_rot = np.array(p.getMatrixFromQuaternion(joint_orn)).reshape(3, 3)
        cam_direction = cam_rot.dot(self.camera.init_camera_vector)
        cam_up_vec = cam_rot.dot(self.camera.init_up_vector)
        # note: if cam is exactly on joint, getCameraImage freezes sometimes
        cam_pos = joint_pos - (cam_direction * 1.0)
        cam_target = joint_pos + (cam_direction * 1.0)
        view_mat = p.computeViewMatrix(cam_pos, cam_target, cam_up_vec)

        return p.getCameraImage(self.camera.pixel_width,
                                self.camera.pixel_height,
                                viewMatrix=view_mat,
                                projectionMatrix=self.camera.projection_mat,
                                flags=p.ER_NO_SEGMENTATION_MASK)

    @staticmethod
    def count_red_pixels(img):
        """detect left red and right red like nrp
        pixelformat is guessed, but it seems to work.
        this functions tries to reproduce https://developer.humanbrainproject.eu/docs/projects/hbp-nrp-cle/1.1.5/codedoc/hbp_nrp_cle.tf_framework.html#hbp_nrp_cle.tf_framework.tf_lib.detect_red"""
        w = img[0]  # width of the image, in pixels
        h = img[1]  # height of the image, in pixels
        rgb_buffer = img[2]  # color data RGB
        # dep_buffer = img_arr[3]  # depth data
        count_red_right = 0
        count_red_left = 0
        count_non_red = 0
        for y in range(h):
            for x in range(w):
                # numpy image
                r, g, b, a = (rgb_buffer[y][x][0], rgb_buffer[y][x][1], rgb_buffer[y][x][2], rgb_buffer[y][x][3])
                if r > g and r > b:
                    if x > w / 2:
                        count_red_right = count_red_right + 1
                    else:
                        count_red_left = count_red_left + 1
                else:
                    count_non_red = count_non_red + 1

        total = count_red_left + count_red_right + count_non_red
        return count_red_left/total, count_red_right/total, count_non_red/total


### run experiment

now that we have defined everything we need, lets boot up the neurosim

In [ ]:
#initial neuro sim

sim.setup(timestep=0.1,min_delay=0.1,max_delay=4.0)
population=get_pop()
sim.initialize(population, v=population.get('v_rest'))
sg = spike_generator(population)
og = output_generator(population)
# population.record('v')
# population.record('spikes')

... and the physic

In [ ]:
# initiate physics

physicsClient = p.connect(p.DIRECT)  
world = World()
robot = Husky()
robot.maxForce = 150
robot.targetVelocity = 20

and now lets to a single step of the simulation. We wrap it in a function, so we can repeat the step at later points

In [ ]:
def step():
    world.do_step()
    robot_img = robot.update_cam()
    count_result = robot.count_red_pixels(robot_img)
    sg.set_rates_from_ratios(*count_result)
    sim.run(10.0)
    voltage = og.get_current_voltage()
    # todo: get better numbers from experiment
    commands = np.multiply(np.subtract(voltage, 0.0002),4000)
    robot.update_control(commands[0].item(), commands[1].item() )
    return og.left,og.right, robot_img
    
step()
print("done")

that was fun, wasn't it? 

So lets do that again 100 times more

In [ ]:
%timeit  -n20 -r5 step()

Now lets take a look on what's actually going on in the experiment. 

We will now set up 3 plots:

* the first shows what the robot sees
* the second shows how the world outside the robot looks like
* the third shows the output-voltage of the brain

In [ ]:
# setup plots, and animate closed loop

from matplotlib import pyplot as plt
from matplotlib import animation
from pyNN.utility.plotting import Figure, Panel
from quantities import mV

%matplotlib widget


# get some initial values by doing a single iteration
output_left,output_right,robot_img = step()
vm_left = output_left.get_data().segments[0].filter(name="v")[0]
vm_right = output_right.get_data().segments[0].filter(name="v")[0]

world_view_mat = p.computeViewMatrix([-5, 5, 5], [1, 0, -3], [0, 0, 1])
world_projection_mat = p.computeProjectionMatrixFOV(fov=80, aspect=1.0/1.0, nearVal=0.1, farVal=100)
world_img = p.getCameraImage(100,100, viewMatrix = world_view_mat, projectionMatrix=world_projection_mat)


# setup plots for physic
fig = plt.figure()
plt.subplot(221)
robot_plot = plt.imshow(robot_img[2], interpolation='none')
plt.subplot(222)
world_plot = plt.imshow(world_img[2], interpolation='none')


# setup plots for brain
x = plt.subplot(2, 1, 2)

plt_left = plt.plot(np.array(vm_left[:,0]))
plt.ylabel("%s (%s)" % (vm_left.name, vm_left.units._dimensionality.string))
plt.setp(plt.gca().get_xticklabels(), visible=True)

plt_right = plt.plot(np.array(vm_right[:,0]))
plt.ylabel("%s (%s)" % (vm_right.name, vm_right.units._dimensionality.string))
plt.setp(plt.gca().get_xticklabels(), visible=True)


def animate(i):    
    output_left,output_right,robot_img = step()
    
    update_voltage_plot(output_left,output_right)
    update_physic_plot(robot_img)

def update_physic_plot(robot_img):
    robot_plot.set_data(robot_img[2])
    world_img = p.getCameraImage(100,100, viewMatrix = world_view_mat, projectionMatrix=world_projection_mat)
    world_plot.set_data(world_img[2])
    

def update_voltage_plot(left,right):
    global vm_left, vm_right
    vm_left_new = left.get_data(clear=True).segments[0].filter(name="v")[0]
    vm_right_new = right.get_data(clear=True).segments[0].filter(name="v")[0]
    a = np.array(vm_left_new[:,0])
    b = np.array(vm_right_new[:,0])
    vm_left = np.concatenate((np.array(vm_left), a))
    vm_right = np.concatenate((np.array(vm_right), b))
    mask = np.zeros(len(vm_left), dtype=bool)
    mask[-2000:-1:] = True
    vm_left = vm_left[mask]
    
    mask = np.zeros(len(vm_right), dtype=bool)
    mask[-2000:-1:] = True
    vm_right = vm_right[mask]
    
    plt_left[0].set_data(np.arange(vm_left.size),vm_left)
    plt_right[0].set_data(np.arange(vm_right.size),vm_right)
    x.relim()
    x.autoscale_view(True,True,True)


plt.show()
anim1 = animation.FuncAnimation(fig, animate, repeat = True, interval=10)


to start the animatio, pleae run the last cell manually once (ctrl+enter)